In [5]:
import pandas as pd
import neptune

NEPTUNE_API_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=='

PROJECT_ID = 'revivemed/Survival-RCC'

In [6]:
df_file = '/Users/jonaheaton/Desktop/Survival-RCC (4).csv'
df = pd.read_csv(df_file)
# chosen_columns = ['desc_str','sweep_kwargs/batch_size','sweep_kwargs/encoder_weight']

In [7]:
start_columns = df.columns
start_columns = start_columns.drop(['Creation Time','Tags','Id'])
start_columns = start_columns.to_list()
val_columns = []
test_columns = []
param_columns = []
new_columns = []

for x in start_columns:
    if ('OS' in x) and ('avg_training_run' in x):
        new_x = x.replace('OS','PFS')
        new_columns.append(new_x)

for x in start_columns:
    if ('IMDC' in x) and ('avg_training_run' in x):
        new_x = x.replace('IMDC','MSKCC')
        # new_columns.append(new_x)

# oops I still need to compute the pfs columns
# start_columns = start_columns + new_columns
# start_columns = list(set(start_columns))


In [8]:
for x in start_columns:
    if 'avg_training_run' in x:
        val_columns.append(x)
        new_x = x.replace('avg_training_run/','avg_testing_run/')
        if 'val__' in new_x:
            new_x = new_x.replace('val__','test__')
            test_columns.append(new_x)
        else:
            test_columns.append(new_x)
    else:
        param_columns.append(x)

chosen_columns = param_columns + val_columns + test_columns

In [9]:
project = neptune.init_project(
    project=PROJECT_ID, 
    mode="read-only",
)

runs_table_df = project.fetch_runs_table(
    query='(`avg_training_run/num_success`:int >= 5)',
    columns = chosen_columns,
).to_pandas()

if 'sys/id' not in param_columns:
    param_columns = ['sys/id']	+ param_columns

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/


Fetching table...: 0 [00:00, ?/s]

In [10]:
df = runs_table_df[param_columns + val_columns + test_columns].copy()

In [11]:
# create a two level column index
# first level is the type of data (param, val, test)
# second level is the column name

df[val_columns] = df[val_columns].round(3)
df[test_columns] = df[test_columns].round(3)

# create a list of tuples for the columns
skip_std = True

columns = []
skip_cols = []
for x in param_columns:
    x0 = x.replace('sweep_kwargs/','')
    x0 = x0.replace('_',' ')
    columns.append(('param',x0))

for x in val_columns:
    x0 = x.replace('avg_training_run/','')
    x0 = x0.replace('metrics/', '')
    x0 = x0.replace('val__','')
    x0 = x0.replace('_Concordance Index', ' C-Index')
    x0 = x0.replace('_AUROC (ovo, macro)', ' AUC')
    x0 = x0.replace('__', ' ')
    x0 = x0.replace('head_','head[')
    x0 = x0.replace(' on_', '] on ')
    # x0 = x0.replace('on_', ' on ')
    x0 = x0.replace('num_success','num iterations')

    if skip_std and 'std' in x0[:4]:
        skip_cols.append(x)
        continue
    columns.append(('val',x0))

for x in test_columns:
    x0 = x.replace('avg_testing_run/','')
    x0 = x0.replace('metrics/', '')
    x0 = x0.replace('test__','')
    x0 = x0.replace('_Concordance Index', ' C-Index')
    x0 = x0.replace('_AUROC (ovo, macro)', ' AUC')
    x0 = x0.replace('__', ' ')
    x0 = x0.replace('head_','head[')
    x0 = x0.replace(' on_', '] on ')
    # x0 = x0.replace('on_', ' on ')
    x0 = x0.replace('num_success','num iterations')
    if skip_std and 'std' in x0[:4]:
        skip_cols.append(x)
        continue
    columns.append(('test',x0))

# create a multiindex
columns = pd.MultiIndex.from_tuples(columns)

# set the columns
kept_cols = [x for x in df.columns if x not in skip_cols]
df = df[kept_cols].copy()
df.columns = columns

# round the values in the val and test columns to three decimal places
# df = df.round({('val',x):3 for x in val_columns})
# df = df.round({('test',x):3 for x in test_columns})

# clean up the 

In [12]:
# save to excel
df.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered.xlsx')

In [13]:
val_ressum = df.groupby([('param','desc str'),('param','name'),('param','use rand init')])['val'].sum()#.unstack(fill_value=0)
test_ressum = df.groupby([('param','desc str'),('param','name'),('param','use rand init')])['test'].sum()#.unstack(fill_value=0)

val_res = df.groupby([('param','desc str'),('param','name'),('param','use rand init')])['val'].mean()#.unstack(fill_value=0)
test_res = df.groupby([('param','desc str'),('param','name'),('param','use rand init')])['test'].mean()#.unstack(fill_value=0)
val_res = val_res.round(3)
test_res = test_res.round(3)

val_res[('val','num iterations')] = val_ressum[('val','num iterations')].copy()
test_res[('test','num iterations')] = test_ressum[('test','num iterations')].copy()
val_res.index.names = ['task desc', 'arch ID', 'used rand init']
test_res.index.names = ['task desc', 'arch ID', 'used rand init']
both_res = pd.concat([val_res, test_res], axis=1)

both_res.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered-agg.xlsx')

In [20]:
val_res

val  \
                                     num iterations   
task desc arch ID     used rand init                  
Both-OS   basic 1.0   False                      35   
                      True                       30   
          basic 1.2   False                      50   
                      True                       40   
          basic 2.1   False                      20   
...                                             ...   
both-PFS  layer-R 1.1 True                       10   
          layer-R 1.2 False                      10   
                      True                       10   
          layer-R 2.0 False                      10   
                      True                       10   

                                                                       \
                                     head[Both OS] on Both OS C-Index   
task desc arch ID     used rand init                                    
Both-OS   basic 1.0   False                                     0.714   
                      True                                      0.696   
          basic 1.2   False                                     0.715   
                      True                                      0.698   
          basic 2.1   False                                     0.716   
...                                                               ...   
both-PFS  layer-R 1.1 True                                        NaN   
          layer-R 1.2 False                                       NaN   
                      True                                        NaN   
          layer-R 2.0 False                                       NaN   
                      True                                        NaN   

                                                                       \
                                     head[Both OS] on NIVO OS C-Index   
task desc arch ID     used rand init                                    
Both-OS   basic 1.0   False                                     0.720   
                      True                                      0.708   
          basic 1.2   False                                     0.722   
                      True                                      0.712   
          basic 2.1   False                                     0.722   
...                                                               ...   
both-PFS  layer-R 1.1 True                                        NaN   
          layer-R 1.2 False                                       NaN   
                      True                                        NaN   
          layer-R 2.0 False                                       NaN   
                      True                                        NaN   

                                                                       \
                                     head[Both OS] on EVER OS C-Index   
task desc arch ID     used rand init                                    
Both-OS   basic 1.0   False                                     0.714   
                      True                                      0.696   
          basic 1.2   False                                     0.714   
                      True                                      0.694   
          basic 2.1   False                                     0.718   
...                                                               ...   
both-PFS  layer-R 1.1 True                                        NaN   
          layer-R 1.2 False                                       NaN   
                      True                                        NaN   
          layer-R 2.0 False                                       NaN   
                      True                                        NaN   

                                                                       \
                                     head[NIVO OS] on Both OS C-Index   
task desc arch ID     used rand init                          

In [32]:
reshaped_df = both_res.unstack('used rand init')

In [33]:
new_cols = []
for col in reshaped_df.columns:
    x = col[1]
    if 'head' in x:
        x_split = x.split(' on ')
        x_p1 = x_split[0]
        x_p2 = x_split[1]
    else:
        x_p1 = x
        x_p2 = x

    if col[2] == True:
        new_x = (col[0], x_p1, x_p2, 'rand init')
    else:
        new_x = (col[0], x_p1, x_p2, 'pretrained')

    # new_x = (col[0], x_p1, x_p2, col[2])
    new_cols.append(new_x)

reshaped_df.columns = pd.MultiIndex.from_tuples(new_cols)

In [34]:
reshaped_df.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered-agg-reshaped.xlsx')

In [105]:
for task_id in both_res.index.levels[0]:
    task_df = both_res.loc[task_id].copy()
    # drop the columns with nans
    task_df = task_df.dropna(axis=1, how='all')
    task_df.to_excel(f'/Users/jonaheaton/Desktop/Survival-RCC-Filtered-agg-{task_id}.xlsx')
    print(f'Saved {task_id} to excel')

Saved Both-OS to excel
Saved EVER-OS to excel
Saved EVER-PFS to excel
Saved IMDC to excel
Saved IMDC-multi to excel
Saved IMDC-ord to excel
Saved MSKCC to excel
Saved MSKCC-multi to excel
Saved MSKCC-ord to excel
Saved NIVO-OS to excel
Saved NIVO-OS ADV EVER-OS to excel
Saved NIVO-OS AND EVER-OS to excel
Saved NIVO-PFS to excel
Saved both-OS AUX Benefit to excel
Saved both-PFS to excel


In [106]:
val_ressum = df.groupby([('param','name'),('param','desc str'),('param','use rand init')])['val'].sum()#.unstack(fill_value=0)
test_ressum = df.groupby([('param','name'),('param','desc str'),('param','use rand init')])['test'].sum()#.unstack(fill_value=0)

val_res = df.groupby([('param','name'),('param','desc str'),('param','use rand init')])['val'].mean()#.unstack(fill_value=0)
test_res = df.groupby([('param','name'),('param','desc str'),('param','use rand init')])['test'].mean()#.unstack(fill_value=0)
val_res = val_res.round(3)
test_res = test_res.round(3)

val_res[('val','num iterations')] = val_ressum[('val','num iterations')].copy()
test_res[('test','num iterations')] = test_ressum[('test','num iterations')].copy()
val_res.index.names = ['arch ID','task desc','used rand init']
test_res.index.names = val_res.index.names
both_res = pd.concat([val_res, test_res], axis=1)

both_res.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered-agg2.xlsx')

In [114]:
# select the rows that have 'OS' in the name
df1 = both_res[both_res.index.get_level_values('task desc').str.contains('OS')].copy()
df2 = df1[df1.index.get_level_values('arch ID').str.contains('basic 1.2')].copy()
df2 = df2.dropna(axis=1, how='all')

df2.to_excel('/Users/jonaheaton/Desktop/Survival-RCC-Filtered-agg2-OS-basic-1.2.xlsx')

Exception in thread NeptuneWebhooks:
Traceback (most recent call last):
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_engine/lib/python3.9/site-packages/websocket/_socket.py", line 115, in recv
    bytes_ = _recv()
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_engine/lib/python3.9/site-packages/websocket/_socket.py", line 94, in _recv
    return sock.recv(bufsize)
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_engine/lib/python3.9/ssl.py", line 1260, in recv
    return self.read(buflen)
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_engine/lib/python3.9/ssl.py", line 1135, in read
    return self._sslobj.read(len)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_engine/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/jonaheaton/opt/miniconda3/envs/mz_embed_eng

## Assign names to architectures

In [ ]:
layer1_0 = {
    'noise_factor': 0.25,
    'learning_rate': 0.0007869775056037999,
    'l2_reg_weight': 1.0092405183765013e-05,
    'l1_reg_weight': 3.137204254745065e-05,
    'num_epochs': 87,
    'head_hidden_layers': 1,
    'EVER-OS__weight': 10.0,
    'NIVO-OS__weight': 5.0,
    'name': 'layer 1.0'
}

list_layerS = [layer1_0]

############## Layer-R methods ############

layerR1_0 = {
    'noise_factor': 0.1,
    'learning_rate': 0.0006221023998363983,
    'l2_reg_weight': 0.0,
    'l1_reg_weight': 0.0025635844524779894,
    'num_epochs': 93,
    'encoder_weight': 1.0,
    'dropout_rate': 0.3,
    'adversarial_start_epoch': 0,
    'adversary_weight': 1.0,
    'head_hidden_layers': 1,
    'EVER-OS__weight': 10.0,
    'NIVO-OS__weight': 4.0,
    'name': 'layer-R 1.0'
}

list_layerR = [layerR1_0]


######## Basic methods ############

basic1_0 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 64,
    'head_hidden_layers': 0,
    'name' : 'basic 1.0'
}

basic1_1 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'name': 'basic 1.1'
}

basic1_2 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.000541422,
    'noise_factor': 0.2,
    'num_epochs': 71,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'remove_nans': True,
    'name': 'basic 1.2'
}

basic2_0 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.0013,
    'noise_factor': 0.2,
    'num_epochs': 98,
    'batch_size': 32,
    'clean_batch': True,
    'head_hidden_layers': 0,
    'weight_decay': 0.000016,
    'name': 'basic 2.0'
}

basic2_1 = {
    'dropout_rate': 0.0,
    'encoder_weight': 0.0,
    'learning_rate': 0.0013,
    'noise_factor': 0.2,
    'num_epochs': 98,
    'batch_size': 32,
    'clean_batch': False,
    'head_hidden_layers': 0,
    'weight_decay': 0.000016,
    'remove_nans': True,
    'name': 'basic 2.1'
}

list_basicS = [basic1_0, basic1_1, basic1_2, basic2_0, basic2_1]

###### Basic-R methods ############

basicR1_0 = {
    'dropout_rate': 0.4,
    'encoder_weight': 0.75,
    'learning_rate': 0.0021,
    'noise_factor': 0.25,
    'num_epochs': 38,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'weight_decay': 0.0,
    'remove_nans': True,
    'name': 'basic-R 1.0'
}

basicR1_1 = {
    'dropout_rate': 0.4,
    'encoder_weight': 0.75,
    'learning_rate': 0.0021,
    'noise_factor': 0.25,
    'num_epochs': 38,
    'batch_size': 32,
    'head_hidden_layers': 0,
    'weight_decay': 0.0,
    'remove_nans': False,
    'name': 'basic-R 1.1'
}

list_basicR = [basicR1_0, basicR1_1]



all_methods = list_layerS + list_layerR + list_basicS + list_basicR

In [ ]:
for with_id in df[('param','sys/id')].to_list():

    run = neptune.init_run(project=PROJECT_ID,
                            api_token=NEPTUNE_API_TOKEN,
                            with_id=with_id)

    run_sweep = run['sweep_kwargs'].fetch()
    matches_name_list = []
    for method in all_methods:
        does_match = True

        for key,val in method.items():
            if key == 'name':
                continue
            if key in run_sweep:
                if run_sweep[key] != val:
                    does_match = False
        
        if does_match:
            print('we have a match to method', method['name'])
            run['sweep_kwargs/name'] = method['name']
            matches_name_list.append(method['name'])

    if len(matches_name_list) < 1:
        print(f'no match for {with_id}')

    run.stop()

    # break

In [ ]:
list_runs = ['SUR-2640',
'SUR-2639',
'SUR-2638',
'SUR-2637',
'SUR-2634',
'SUR-2633',
'SUR-2632',
'SUR-2631',
'SUR-2630',
'SUR-2629']


for with_id in list_runs:

    run = neptune.init_run(project=PROJECT_ID,
                            api_token=NEPTUNE_API_TOKEN,
                            with_id=with_id)
    
    run['sweep_kwargs/name'] = 'basic-R 1.0'
    run.stop()

In [4]:
df['Id'].to_list()

['SUR-2815',
 'SUR-2814',
 'SUR-2813',
 'SUR-2812',
 'SUR-2811',
 'SUR-2810',
 'SUR-2809',
 'SUR-2808',
 'SUR-2807',
 'SUR-2806',
 'SUR-2805',
 'SUR-2804',
 'SUR-2803',
 'SUR-2802',
 'SUR-2801',
 'SUR-2800',
 'SUR-2799',
 'SUR-2798',
 'SUR-2797',
 'SUR-2796',
 'SUR-2795',
 'SUR-2794',
 'SUR-2793',
 'SUR-2791',
 'SUR-2790',
 'SUR-2789',
 'SUR-2788',
 'SUR-2787',
 'SUR-2786',
 'SUR-2785',
 'SUR-2784',
 'SUR-2783',
 'SUR-2782',
 'SUR-2781',
 'SUR-2780',
 'SUR-2779',
 'SUR-2778',
 'SUR-2777',
 'SUR-2776',
 'SUR-2775',
 'SUR-2774',
 'SUR-2773',
 'SUR-2772',
 'SUR-2771',
 'SUR-2770',
 'SUR-2769',
 'SUR-2768',
 'SUR-2767',
 'SUR-2766',
 'SUR-2765',
 'SUR-2764',
 'SUR-2763',
 'SUR-2762',
 'SUR-2761',
 'SUR-2760',
 'SUR-2759',
 'SUR-2758',
 'SUR-2757',
 'SUR-2756',
 'SUR-2755',
 'SUR-2754',
 'SUR-2753',
 'SUR-2752',
 'SUR-2751',
 'SUR-2750',
 'SUR-2749',
 'SUR-2748',
 'SUR-2747',
 'SUR-2746',
 'SUR-2745',
 'SUR-2744',
 'SUR-2743',
 'SUR-2742',
 'SUR-2741',
 'SUR-2740',
 'SUR-2739',
 'SUR-2738',

In [6]:
finished_ids = []

for with_id in df['Id'].to_list():
    if with_id == 'SUR-2808':
        break
    finished_ids.append(with_id)

In [7]:
finished_ids

['SUR-2815',
 'SUR-2814',
 'SUR-2813',
 'SUR-2812',
 'SUR-2811',
 'SUR-2810',
 'SUR-2809']

In [3]:
from main_finetune import finetune_run_wrapper

finished_ids = []
'SUR-2808'

for with_id in df['Id'].to_list():
# for with_id in df[('param','sys/id')].to_list():
    finetune_run_wrapper(with_id=with_id)
    finished_ids.append(with_id)

Getting the latest dataset
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/metadata


[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-2815
eval_params_list [{'y_col_name': 'NIVO OS', 'y_head': 'NIVO OS', 'y_cols': ['NIVO OS', 'OS_Event']}, {'y_col_name': 'EVER OS', 'y_head': 'NIVO OS', 'y_cols': ['EVER OS', 'OS_Event']}, {'y_col_name': 'Both OS', 'y_head': 'NIVO OS', 'y_cols': ['OS', 'OS_Event']}, {'y_col_name': 'IMDC BINARY', 'y_head': 'IMDC', 'y_cols': ['IMDC BINARY']}, {'y_col_name': 'IMDC ORDINAL', 'y_head': 'IMDC_Ordinal', 'y_cols': ['IMDC ORDINAL']}, {'y_col_name': 'IMDC ORDINAL', 'y_head': 'IMDC_MultiClass', 'y_cols': ['IMDC ORDINAL']}]
Using Neptune


Fetching file...: 0 [00:00, ?/s]

Error: Invalid head name: NIVO OS
Error: Invalid head name: NIVO OS
Error: Invalid head name: NIVO OS
Error: Invalid head name: IMDC
Error: Invalid head name: IMDC_Ordinal
Error: Invalid head name: IMDC_MultiClass
Error: Invalid head name: NIVO OS
Error: Invalid head name: NIVO OS
Error: Invalid head name: NIVO OS
Error: Invalid head name: IMDC
Error: Invalid head name: IMDC_Ordinal
Error: Invalid head name: IMDC_MultiClass
Using Neptune


...

Error: Invalid head name: MSKCC_Ordinal
Error: Invalid head name: MSKCC_MultiClass
Error: Invalid head name: MSKCC_Ordinal
Error: Invalid head name: MSKCC_MultiClass
Using Neptune


Fetching file...: 0 [00:00, ?/s]

Error: Invalid head name: MSKCC_Ordinal
Error: Invalid head name: MSKCC_MultiClass
Error: Invalid head name: MSKCC_Ordinal
Error: Invalid head name: MSKCC_MultiClass
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 11 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 11 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/e/SUR-2809/metadata
Getting the latest dataset
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/metadata
[neptune] [

Fetching file...: 0 [00:00, ?/s]

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [5]:
with_id

'SUR-2808'